In [ ]:
# https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AmazonS3.html
# https://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Bucket.download_file
# https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AmazonS3.html
# https://stackoverflow.com/questions/42090830/use-boto3-to-download-from-public-bucket

FILE SIZE OPTIONS

In [ ]:
    124361 georgia_shuffled.json 520M
    
    100000 georgia_1e5.json      418M  80.40%  FAIL
     50000 georgia_5e4.json      209M  40.20%  562M
     25000 georgia_25e3.json     105M  20.10%  260M
     12436 georgia_12e3.json      58M  10.00%   64M(?)   
     10000 georgia_1e4.json       42M   8.04%   75M
      5000 georgia_5e3.json       21M   4.02%   41M
      2500 georgia_25e2.json      11M   2.01%   16M  
      1000 georgia_1e3.json      6.1M   0.80%    3M
       500 georgia_5e2.json      3.0M   0.40%    0M
       250 georgia_25e1.json     1.5M   0.20%    0M 

    318611 total

In [1]:
import simplejson as json
import pprint

In [2]:
def print_json(json_obj, skipkeys=False, indent=2, sort_keys=False):
    
    print json.dumps(json_obj, skipkeys=skipkeys, indent=indent, sort_keys=sort_keys)

# print_json(georgia[0], indent=4, sort_keys=True)

#### LOAD FILE

In [3]:
# # HELPFUL LINKS:
# # 1. http://zetcode.com/python/simplejson/  


databases = 'data/databases/'
state = 'georgia/'
file_name = 'georgia_25e1.json'

with open(databases + state + file_name) as json_file:
    georgia = json.load(json_file)

In [99]:
# # # SPATIAL
# # {"county": "Chatham County", "country": "United States", "state": "Georgia", 
# #  "name": "Savannah (Ga.)","coordinates": "32.08354, -81.09983"}

# # # STATELOCATEDIN
# # {"name": "Athens, Ga. : Digital Library of Georgia"}

spatial_keys = ['county', 'country', 'state', 'name', 'coordinates']

try:
    spatial = georgia[5]['_source']['sourceResource']['spatial'][0]
    
    for spatial_key in spatial_keys:
#         print spatial_key
        
        if spatial_key in spatial:
            exists = True
        else:
            exists = False
            
        print spatial_key + ':', exists, '\t',

#     print_json(spatial, indent=2)

except Exception as err:
    
    print err.__class__.__name__ + ' ' + str(err)
#     print traceback

county: True 	country: True 	state: True 	name: True 	coordinates: True 	

#### SPATIAL

In [121]:
count = 0
spatial_keys = ['county', 'country', 'state', 'name', 'coordinates']

for i, record in enumerate(georgia):
    
    try:
        
        sourceResource = record['_source']['sourceResource']
        spatial = sourceResource['spatial'][0]
        
#         print str(i) + '.',
        
#         for spatial_key in spatial_keys:
        
#             if spatial_key in spatial:
#                 exists = True
#             else:
#                 exists = False

#             print spatial_key + ':', exists, '\t',
#         print ''
        if spatial['coordinates']:
            print str(i) +'.', spatial['coordinates']
        
    except Exception as err:
        print str(i)+'.', err.__class__.__name__ + ': ' + str(err)
#         count += 1

# print count

In [ ]:
NO SPATIAL INFORMATION: 15
5. KeyError 'spatial'


COUNTY: 207
Name + County

NO COUNTY: 28
89. KeyError: 'county'
    
    
COUNTRY:
9. United States (231)
10. Canada (1)
    
NO COUNTRY: 3
119. KeyError: 'country'

    
STATE: 229
0. Georgia: (223)
10. Yukan (1)
47. Mississippi (1)
89. Florida (3)
150. Ohio (1)

NO STATE: 6
240. KeyError: 'state'

    
NAME:
0. Savannah (Ga.)
3. United States
4. Georgia
10. Dawson (Yukon)
11. Gordon County (Ga.)
45. Chicago (Ill.)
47. Mississippi
112. Houston Street (Atlanta, Ga.)
119. Atlanta Metropolitan Area (Ga.)
194. Dalton (Whitfield County, Ga.)

NO NAME: 0


COORDINATES:
0. 32.08354, -81.09983

NO COORDINATES: 6

#### STATE LOCATED IN

In [88]:
count = 0

for i, record in enumerate(georgia):
    
    try:
        
        sourceResource = record['_source']['sourceResource']
        stateLocatedIn = sourceResource['stateLocatedIn'][0]

        if not (':' in stateLocatedIn['name']):
            print str(i) +'.', stateLocatedIn['name']
#             count += 1
    except Exception as err:
        print str(i)+'.', err.__class__.__name__ + ': ' + str(err)
#         count += 1

# print count

In [ ]:
STATE EXPLICIT: 200
0. Athens, Ga. : Digital Library of Georgia
6. Savannah (Ga.) : Asa H. Gordon Library in association with the Internet Archive
47. Jackson (Miss.)] : Mississippi Dept. of Archives and History
117. Athens, Ga.] : Digital Library of Georgia in association with the Troup County Archives
174. Augusta (Ga.) : Reese Library in association with the Internet Archive

STATE INFERRED: 34
1. Georgia State University Library
246. Special Collections and Archives, Georgia State University Library
111. Kenan Research Center, Atlanta History Center, 130 West Paces Ferry Road, Atlanta, GA 30305

LOCATION IS NOT A STATE: 5
23. Knowledge Box
79. New Georgia Encyclopedia
94. Digital Library of Georgia

NO HOLDING INFORMATION: 11
5. KeyError 'stateLocatedIn'